In [ ]:
import os
import pickle

path = "/media/NLP/simple_manipulation"
train_path = os.path.join(path, "train")
val_path = os.path.join(path, "val")
test_path = os.path.join(path, "test")

train = os.listdir(train_path)
train = train[0:100]
print(train)

In [ ]:
!ls $element_path

In [ ]:
element_path = os.path.join(train_path, train[1])
action_path = os.path.join(element_path, "action.pkl")
assert os.path.exists(action_path)

with open(action_path, "rb") as f:
    action = pickle.load(f)

action

In [ ]:
from pose_quantizer import PoseQuantizer

pq = PoseQuantizer(-0.5, 0.75, 100)
print("Original: " + str(action['pose0_position'][0]))
enc = pq.encode_array(action['pose0_position'][0])
print("Encoded: " + str(enc))
dec = pq.decode_array(enc)
print("Decoded: " + str(dec))

In [ ]:
element_path = os.path.join(train_path, train[1])
trajectory_path = os.path.join(element_path, "trajectory.pkl")
assert os.path.exists(trajectory_path)

with open(trajectory_path, "rb") as f:
    trajectory = pickle.load(f)


trajectory

In [ ]:
for data in train:
    element_path = os.path.join(train_path, data)
    trajectory_path = os.path.join(element_path, "trajectory.pkl")
    assert os.path.exists(trajectory_path)

    with open(trajectory_path, "rb") as f:
        trajectory = pickle.load(f)
    print(trajectory['action_bounds'])

#trajectory

In [7]:
from transformers import T5Tokenizer
import numpy as np
import sys

tokenizer = T5Tokenizer.from_pretrained(
        "t5-base", model_max_length=256, extra_ids=1200) 

res = tokenizer("action: pose: <extra_id_1177><extra_id_1171><extra_id_1143> rotation: <extra_id_1140><extra_id_1140><extra_id_1140><extra_id_1199> action: pose: <extra_id_1177><extra_id_1150><extra_id_1142> rotation: <extra_id_1140><extra_id_1140><extra_id_1199><extra_id_1114>", max_length=27, truncation=True, padding='max_length')

#tokenizer.decode(res)
res
#res = np.array(res)
#print(res)
#masks = np.ones((res.shape[-1], res.shape[-1]))
#masks = np.triu(masks, 1)
#masks


{'input_ids': [1041, 10, 7663, 10, 32022, 32028, 32056, 11981, 10, 32059, 32059, 32059, 32000, 1041, 10, 7663, 10, 32022, 32049, 32057, 11981, 10, 32059, 32059, 32000, 32085, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [2]:
import process_data

path = "/media/NLP/simple_manipulation/train/"
element_path = os.path.join(path, '000000')
process_data.processDataPoint(element_path)

IMAGE SHAPE: (128, 256, 3)
IMAGE TENSOR SHAPE: (384, 384, 3)
[[[ 1.9577874  2.1309524  2.3437037]
  [ 1.9663497  2.1397057  2.3524182]
  [ 2.0091617  2.1834733  2.3959913]
  ...
  [ 2.040557   2.2155695  2.4279447]
  [ 1.9834745  2.1572127  2.3698473]
  [ 1.9064132  2.0784314  2.2914162]]

 [[ 1.9663497  2.1397057  2.3524182]
  [ 1.9535064  2.1265757  2.3393464]
  [ 1.923538   2.0959384  2.3088453]
  ...
  [ 1.5838969  1.7487161  1.9631662]
  [ 1.9363816  2.1090686  2.3219173]
  [ 2.0690982  2.2447479  2.4569933]]

 [[ 2.0063074  2.1805553  2.3930862]
  [ 1.9849015  2.1586719  2.3712997]
  [ 1.903559   2.0755134  2.288511 ]
  ...
  [ 1.1767083  1.3324386  1.5487388]
  [ 1.9135486  2.085726   2.2986784]
  [ 2.2060962  2.384804   2.5964267]]

 ...

 [[-2.117904  -2.0357144 -1.8044444]
  [-2.117904  -2.0357144 -1.8044444]
  [-2.117904  -2.0357144 -1.8044444]
  ...
  [-2.117904  -2.0357144 -1.8044444]
  [-2.117904  -2.0357144 -1.8044444]
  [-2.117904  -2.0357144 -1.8044444]]

 [[-2.117904 

pybullet build time: May 20 2022 19:45:31


[INFO] 17 tasks loaded


In [1]:
import argparse
import wandb

import numpy as np

from tqdm.auto import tqdm

import jax
import jax.numpy as jnp
import optax
from flax.training import train_state, checkpoints
from flax.training.common_utils import shard
from uio import utils
from uio import network
from uio.configs import CONFIGS, VAE_CONFIG
from uio.model import UnifiedIOModel

def init_train_state(
    model, params, learning_rate
) -> train_state.TrainState:
    optimizer = optax.adam(learning_rate)
    return train_state.TrainState.create(
        apply_fn = model.module.apply,
        tx=optimizer,
        params=params
    )

conf = CONFIGS["small"]
module = network.Transformer(config=conf, vae_config=VAE_CONFIG)
module

2022-12-06 15:08:37.903823: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-06 15:08:37.903860: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-06 15:08:37.903864: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


Transformer(
    # attributes
    config = UnifiedIOConfig(vocab_size=33152, image_vocab_size=16384, image_patch_size=16, dtype='float32', emb_dim=512, num_heads=6, num_encoder_layers=8, num_decoder_layers=8, head_dim=64, mlp_dim=1024, mlp_activations=('gelu', 'linear'), dropout_rate=0.0, logits_via_embedding=True, float32_attention_logits=False, encoder_max_image_length=576, encoder_max_text_length=256, decoder_max_image_length=256, decoder_max_text_length=256, visual_backbone_type=None, visual_backbone_feature=None, default_image_size=(384, 384), num_seg_emb=2)
    vae_config = VAEConfig(embed_dim=256, n_embed=16384, double_z=False, z_channels=256, resolution=256, in_channels=3, out_ch=3, ch=128, ch_mult=(1, 1, 2, 2, 4), num_res_blocks=2, attn_resolutions=(16,), dropout=0, dtype='float32')
)

In [2]:
model = UnifiedIOModel(module, text_decoder_length=28, image_decoder_length=1)
model

UnifiedIOModel()

In [3]:
params = utils.load_checkpoint("/home/greg/NLP-Final/unified-io-inference/base.bin")

In [4]:
sum(p.size for p in jax.tree_leaves(params))/10e6

/tmp/ipykernel_35285/1899788700.py:1: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  sum(p.size for p in jax.tree_leaves(params))/10e6


31.7520771

In [5]:
state = init_train_state(model, params, learning_rate=5e-5)#decrease by 20%

2022-12-06 15:10:00.092182: W external/org_tensorflow/tensorflow/compiler/xla/service/gpu/nvptx_compiler.cc:497] The NVIDIA driver's CUDA version is 11.7 which is older than the ptxas CUDA version (11.8.89). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
